In [ ]:
# read the data

from os import walk
f =[]

# get all filenames in the directory
for (a, b, filenames) in walk("data/"):
     f.extend(filenames) 
     break
print (filenames)

# split the filenames
data_contain = {'name': [], 'rank_all': [], 'death_all': [], 'rank_0_14': [], 'death_0_14':[], 'rank_15_24': [],
                      'death_15_24': [], 'rank_25_34': [], 'death_25_34': [], 'rank_35_44': [], 'death_35_44': [], 
                      'rank_45_54': [],'death_45_54': [], 'rank_55_64': [], 'death_55_64': [], 'rank_65_74': [], 
                      'death_65_74': [], 'rank_75': [], 'death_75+': [], 'gender':[],'state':[]}

all_data = pd.DataFrame(data_contain) 
# read the datasheet
gender =['female','male','both']

# add the population to the file, population= dict type 
population = {}; 

for g in gender:
    population[g] = pd.read_excel("population.xlsx",sheet_name=g)
    population[g].set_index("state", drop=True, append=False,inplace=True)

for  filename in filenames:
    if filename.split(".")[0] == '':
        continue
    path = "data/"+ filename

    # read the state data
    for g in gender:
        state_name = filename.split(".")[0] # generate state name by using filename {statename.xlsx}
        state_data = pd.read_excel (path,sheet_name=g) # read state data
        disease_number = state_data.shape[0] # the row number for this state data = the number of the disease
        state_data['gender'] = [g]* disease_number 
        state_data['state'] = [state_name]* disease_number
        
        state_population= pd.DataFrame(np.repeat((population[g].loc[[state_name],:]).values,disease_number,axis=0)) # read the population data
        state_population.columns = population[g].columns
        state_data = state_data.join(state_population) # join state_data with population data
        
        all_data = all_data.append (state_data,ignore_index = True, sort=None)
        
# save to csv
all_data.to_csv("state_population.csv", index=False)

